## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [25]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [26]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [27]:
# Set up data path
data_path = 'data'

In [28]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,7/8/18,CP BL,7.04,NaN,47.0,0.78,8.99,Specialized
1,10/12/16,Citibike from Work,0.84,NaN,7.0,0.12,7.20,Citibike
2,8/23/16,Citibike to Work,1.46,NaN,15.0,0.25,5.84,Citibike
3,12/2/16,Citibike from Work,0.92,NaN,7.0,0.12,7.89,Citibike
4,8/4/16,Citibike to Work,1.40,NaN,11.0,0.18,7.64,Citibike
...,...,...,...,...,...,...,...,...
365,12/24/20,Dn ES Up LWS,13.79,1.0,41.0,1.68,8.19,Specialized
366,6/23/20,UWS Thru CP to GWB+,22.42,2.0,54.0,2.90,7.73,Specialized
367,10/5/16,Citibike to Work Midday,1.16,NaN,13.0,0.22,5.35,Citibike
368,12/11/20,Dn ES Up WS,21.60,2.0,36.0,2.60,8.31,Specialized


In [29]:
# Change NaN values to 0 - only occurs on hours column so this is accurate
all_df['Hours'] = all_df['Hours'].fillna(0)
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,7/8/18,CP BL,7.04,0.0,47.0,0.78,8.99,Specialized
1,10/12/16,Citibike from Work,0.84,0.0,7.0,0.12,7.20,Citibike
2,8/23/16,Citibike to Work,1.46,0.0,15.0,0.25,5.84,Citibike
3,12/2/16,Citibike from Work,0.92,0.0,7.0,0.12,7.89,Citibike
4,8/4/16,Citibike to Work,1.40,0.0,11.0,0.18,7.64,Citibike
...,...,...,...,...,...,...,...,...
365,12/24/20,Dn ES Up LWS,13.79,1.0,41.0,1.68,8.19,Specialized
366,6/23/20,UWS Thru CP to GWB+,22.42,2.0,54.0,2.90,7.73,Specialized
367,10/5/16,Citibike to Work Midday,1.16,0.0,13.0,0.22,5.35,Citibike
368,12/11/20,Dn ES Up WS,21.60,2.0,36.0,2.60,8.31,Specialized


In [30]:
# Turning Index into Orig_Index Column
all_df.reset_index(inplace=True)
all_df = all_df.rename(columns = {'index': 'Orig_Index'})
all_df

,Orig_Index,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,0,7/8/18,CP BL,7.04,0.0,47.0,0.78,8.99,Specialized
1,1,10/12/16,Citibike from Work,0.84,0.0,7.0,0.12,7.20,Citibike
2,2,8/23/16,Citibike to Work,1.46,0.0,15.0,0.25,5.84,Citibike
3,3,12/2/16,Citibike from Work,0.92,0.0,7.0,0.12,7.89,Citibike
4,4,8/4/16,Citibike to Work,1.40,0.0,11.0,0.18,7.64,Citibike
...,...,...,...,...,...,...,...,...,...
365,365,12/24/20,Dn ES Up LWS,13.79,1.0,41.0,1.68,8.19,Specialized
366,366,6/23/20,UWS Thru CP to GWB+,22.42,2.0,54.0,2.90,7.73,Specialized
367,367,10/5/16,Citibike to Work Midday,1.16,0.0,13.0,0.22,5.35,Citibike
368,368,12/11/20,Dn ES Up WS,21.60,2.0,36.0,2.60,8.31,Specialized


In [31]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [32]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,7.04,0.78,8.99,Specialized
1,0.84,0.12,7.20,Citibike
2,1.46,0.25,5.84,Citibike
3,0.92,0.12,7.89,Citibike
4,1.40,0.18,7.64,Citibike
...,...,...,...,...
365,13.79,1.68,8.19,Specialized
366,22.42,2.90,7.73,Specialized
367,1.16,0.22,5.35,Citibike
368,21.60,2.60,8.31,Specialized


In [33]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
# bike_df = all_df[['Miles', 'Duration', 'Speed','Type']]
# bike_df

In [34]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [35]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [36]:
# Checking training data shape
X_train.shape

(277, 3)

In [37]:
# Checking testing data shape
X_test.shape

(93, 3)

In [38]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [39]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [40]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [41]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.956989247311828


In [42]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Citibike
1,Citibike
2,Specialized
3,Specialized
4,Specialized
...,...
88,Citibike
89,Citibike
90,Citibike
91,Citibike


In [43]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
144,Citibike
168,Citibike
77,Specialized
132,Specialized
142,Specialized
...,...
93,Citibike
285,Citibike
4,Citibike
324,Citibike


In [44]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,144,Citibike
1,168,Citibike
2,77,Specialized
3,132,Specialized
4,142,Specialized
...,...,...
88,93,Citibike
89,285,Citibike
90,4,Citibike
91,324,Citibike


In [45]:
# Combining the new test and predition dataframes horizontally for comparison
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,144,Citibike,Citibike
1,168,Citibike,Citibike
2,77,Specialized,Specialized
3,132,Specialized,Specialized
4,142,Specialized,Specialized
...,...,...,...
88,93,Citibike,Citibike
89,285,Citibike,Citibike
90,4,Citibike,Citibike
91,324,Citibike,Citibike


In [46]:
# Identify records where prediction is wrong
pred_errs_df = test_pred_df.loc[test_pred_df['Type'] != test_pred_df['Prediction']]
pred_errs_df

,Orig_Index,Type,Prediction
12,78,Specialized,Citibike
43,94,Specialized,Citibike
50,149,Specialized,Citibike
78,338,Specialized,Citibike


In [47]:
# Merging pred_errs_df with all_df to see details of errors
err_details_df = pd.merge(pred_errs_df, all_df, on=["Orig_Index", "Orig_Index"])
err_details_df

,Orig_Index,Type_x,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type_y
0,78,Specialized,Citibike,6/28/20,Downtown Trio,4.10,0.0,34.0,0.57,7.24,Specialized
1,94,Specialized,Citibike,12/29/20,From Trek WS,3.75,0.0,26.0,0.43,8.65,Specialized
2,149,Specialized,Citibike,5/25/20,Uptown Trio,4.27,0.0,34.0,0.57,7.54,Specialized
3,338,Specialized,Citibike,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54,Specialized


In [48]:
# Cleaning up err_details_df to eliminate redundancy and improve readability
err_details_df.drop(columns='Type_y', inplace=True)
err_details_df.rename(columns = {'Type_x' : 'Type'}, inplace=True)
err_details_df

,Orig_Index,Type,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed
0,78,Specialized,Citibike,6/28/20,Downtown Trio,4.10,0.0,34.0,0.57,7.24
1,94,Specialized,Citibike,12/29/20,From Trek WS,3.75,0.0,26.0,0.43,8.65
2,149,Specialized,Citibike,5/25/20,Uptown Trio,4.27,0.0,34.0,0.57,7.54
3,338,Specialized,Citibike,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54
